In [1]:

import pandas as pd
import pickle
import networkx as nx
import sys
import os

sys.path.append(os.path.abspath('..'))
from src.features import get_pagerank, get_approx_betweenness, get_clustering_coefficient

# 1. Load the graph from the pickle file
# Make sure the path is correct relative to where you run this script
with open("../data/processed/amazon_graph.pickle", "rb") as f:
    G_loaded = pickle.load(f)



Configurazione completata. Output sarà:  ../data/processed/amazon_graph.pickle


In [2]:
# 2. Pass the LOADED graph object to the function
pagerank_scores = get_pagerank(G_loaded,0.9)


df_pagerank = pd.DataFrame(list(pagerank_scores.items()), columns=['ASIN', 'PageRank'])
display(df_pagerank.head(5))
# Save to CSV
df_pagerank.to_csv("../data/processed/pagerank_scores.csv", index=False)


# Now you can use the scores
print(f"Computed PageRank for {len(pagerank_scores)} nodes.")

Calcolo PageRank su 334843 nodi...


,ASIN,PageRank
0,0827229534,8.386325e-06
1,0738700797,8.178738e-06
2,0842328327,6.473884e-07
3,1577943082,1.558228e-06
4,0486220125,3.262809e-07


Computed PageRank for 334843 nodes.


In [3]:
#TODO modificare qundo mypagerank sarà nel suo file python
%run mypagerank.ipynb

# Now all functions (build_index_map, calculate_pagerank, etc.) are available in memory
myscores = calculate_pagerank(G_loaded)

df_mypagerank = pd.DataFrame(list(myscores.items()), columns=['ASIN', 'MyPageRank'])
display(df_mypagerank.head(5))
# Save to CSV
df_mypagerank.to_csv("../data/processed/pagerank_scores.csv", index=False)


# Now you can use the scores
print(f"Computed PageRank for {len(myscores)} nodes.")

NO GPU
Building transition matrix...


Running PageRank power iteration...
Converged after 58 iterations.


,ASIN,MyPageRank
0,0827229534,6.480474e-06
1,0738700797,1.074178e-05
2,0842328327,7.111281e-07
3,1577943082,1.594106e-06
4,0486220125,4.578546e-07


Computed PageRank for 334843 nodes.


In [4]:
bet_scores = get_approx_betweenness(G_loaded,1000)


df_bet = pd.DataFrame(list(bet_scores.items()), columns=['ASIN', 'Betweenness'])
display(df_bet.head(5))
# Save to CSV
df_bet.to_csv("../data/processed/bet_scores.csv", index=False)


# Now you can use the scores
print(f"Computed bet cent for {len(bet_scores)} nodes.")

,ASIN,Betweenness
0,0827229534,5.321784e-05
1,0738700797,6.280808e-09
2,0842328327,0.000000e+00
3,1577943082,0.000000e+00
4,0486220125,0.000000e+00


Computed bet cent for 334843 nodes.


In [5]:
clus_scores = get_clustering_coefficient(G_loaded)


df_clus = pd.DataFrame(list(clus_scores.items()), columns=['ASIN', 'ClusteringCoefficient'])
display(df_clus.head(5))
# Save to CSV
df_clus.to_csv("../data/processed/clus_scores.csv", index=False)


# Now you can use the scores
print(f"Computed cc for {len(clus_scores)} nodes.")

,ASIN,ClusteringCoefficient
0,0827229534,0.290323
1,0738700797,0.563953
2,0842328327,0.400000
3,1577943082,0.888889
4,0486220125,0.333333


Computed cc for 334843 nodes.


### Now we work on df


In [6]:
data_frames = [df_pagerank,df_clus,df_bet]

df_final = pd.concat(data_frames,axis=1)

df_final.to_csv("../data/processed/structural_metrics.csv", index=False)

## Clustering

In [11]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

features = ['PageRank','Betweenness','ClusteringCoefficient']
X=df_final[features]

#non ci dovrebbero essere NaN ma per sicurezza
X=X.fillna(0)

Standardize

In [13]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

kmeans = KMeans(n_clusters=4)
clusters = kmeans.fit_predict(X_scaled)

df_final['cluster_kmeans']= clusters

#questo stampa quanti nodi per ogni gruppo
print(df_final['cluster_kmeans'].value_counts())

#maybe we can save it

cluster_kmeans
3    189598
0    120611
2     22373
1      2261
Name: count, dtype: int64


confronto con gruppi 

In [14]:
import seaborn as sns 
import matplotlib.pyplot as plt

crosstab = pd.crosstab(df_final['cluster_kmeans'], df_final['group'])

print(crosstab)

KeyError: 'group'

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(crosstab, annot=True, fmt='d',cmap='YlGnBu')
plt.xlabel('Categoria Originale')
plt.ylabel('Cluster Assegnato')
plt.show()